# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [3]:
import pandas as pd

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df_m = pd.read_csv(url)

In [4]:
df_m.columns = [column.lower().replace(" ","_") for column in df_m.columns]

In [5]:
df_m.nunique()

unnamed:_0                       10910
customer                          9134
state                                5
customer_lifetime_value           8041
response                             2
coverage                             3
education                            5
effective_to_date                   59
employmentstatus                     5
gender                               2
income                            5694
location_code                        3
marital_status                       3
monthly_premium_auto               202
months_since_last_claim             36
months_since_policy_inception      100
number_of_open_complaints            6
number_of_policies                   9
policy_type                          3
policy                               9
renew_offer_type                     4
sales_channel                        4
total_claim_amount                5106
vehicle_class                        6
vehicle_size                         3
vehicle_type             

In [6]:
df_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10910 entries, 0 to 10909
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   unnamed:_0                     10910 non-null  int64  
 1   customer                       10910 non-null  object 
 2   state                          10279 non-null  object 
 3   customer_lifetime_value        10910 non-null  float64
 4   response                       10279 non-null  object 
 5   coverage                       10910 non-null  object 
 6   education                      10910 non-null  object 
 7   effective_to_date              10910 non-null  object 
 8   employmentstatus               10910 non-null  object 
 9   gender                         10910 non-null  object 
 10  income                         10910 non-null  int64  
 11  location_code                  10910 non-null  object 
 12  marital_status                 10910 non-null 

In [9]:
df_m.shape

(10910, 26)

In [10]:
df_m.dropna(how='all',inplace = True )

In [11]:
df_m.shape

(10910, 26)

In [7]:
df_m.duplicated().sum()

0

In [65]:
df_m.isnull().sum()

unnamed:_0                          0
customer                            0
state                             631
customer_lifetime_value             0
response                          631
coverage                            0
education                           0
effective_to_date                   0
employmentstatus                    0
gender                              0
income                              0
location_code                       0
marital_status                      0
monthly_premium_auto                0
months_since_last_claim           633
months_since_policy_inception       0
number_of_open_complaints         633
number_of_policies                  0
policy_type                         0
policy                              0
renew_offer_type                    0
sales_channel                       0
total_claim_amount                  0
vehicle_class                     622
vehicle_size                      622
vehicle_type                     5482
dtype: int64

In [67]:
df_m["state"].unique() 

array(['Arizona', 'California', 'Washington', 'Oregon', nan, 'Nevada'],
      dtype=object)

In [87]:
df_m.dropna(subset =["state"], inplace = True)

In [88]:
df_m.isnull().sum()

unnamed:_0                          0
customer                            0
state                               0
customer_lifetime_value             0
response                            0
coverage                            0
education                           0
effective_to_date                   0
employmentstatus                    0
gender                              0
income                              0
location_code                       0
marital_status                      0
monthly_premium_auto                0
months_since_last_claim           591
months_since_policy_inception       0
number_of_open_complaints         591
number_of_policies                  0
policy_type                         0
policy                              0
renew_offer_type                    0
sales_channel                       0
total_claim_amount                  0
vehicle_class                     591
vehicle_size                      591
vehicle_type                     5164
dtype: int64

Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [89]:
df_m_1k_yes = df_m[ ( df_m.total_claim_amount > 1000) & (df_m.response == "Yes") ]

In [90]:
df_m["response"].unique()

array(['No', 'Yes'], dtype=object)

In [92]:
pivot_df_m_1k_yes = df_m_1k_yes.pivot_table( index="policy_type", columns = "gender", values=['total_claim_amount'], aggfunc='mean', )

In [93]:
pivot_df_m_1k_yes

total_claim_amount             
gender                          F            M
policy_type                                   
Corporate Auto        1138.400000  1171.150007
Personal Auto         1214.853805  1137.861443
Special Auto          1358.400000  1017.500015

Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [94]:
df_m_yes = df_m[ df_m.response == "Yes"]

In [95]:
pivot_df_m_yes = df_m_yes.pivot_table( index="policy_type", columns = "gender", values=['total_claim_amount'], aggfunc='mean', )

In [96]:
pivot_df_m_yes

total_claim_amount            
gender                          F           M
policy_type                                  
Corporate Auto         433.738499  408.582459
Personal Auto          452.965929  457.010178
Special Auto           453.280164  429.527942

Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers

In [108]:
pivot_state2 = df_m_yes.pivot_table( index= "state", values=["customer"], aggfunc='count')

In [109]:
pivot_state2

,customer
state,
Arizona,281
California,504
Nevada,140
Oregon,419
Washington,122


In [110]:
pivot_state = df_m.pivot_table( index= "state", values=["customer"], aggfunc='count')

In [111]:
pivot_state

,customer
state,
Arizona,1937
California,3552
Nevada,993
Oregon,2909
Washington,888


Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [113]:
df_m.groupby('gender')['customer_lifetime_value'].agg(['max', 'mean' , 'mean'])

,max,mean,mean
gender,,,
F,73225.95652,8091.801859,8091.801859
M,83325.38119,7964.362535,7964.362535


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here